In [ ]:
# data_dir="../input/data-art/data_train"

In [ ]:
data_dir="../input/portrait-painting/portrait_painting"

In [ ]:
input_dir="/kaggle/working/datatrain"

In [ ]:
output_dir = "/kaggle/working/Output"

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import time
from PIL import Image
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import models
from keras.models import Sequential, Model, load_model
from keras.layers import Input, Dense, Reshape, Flatten, Activation
from keras.layers import ReLU, LeakyReLU, BatchNormalization
from keras.layers import Conv2D, Conv2DTranspose, UpSampling2D
from keras.optimizers import Adam
from keras.initializers import RandomNormal

from IPython import display
from PIL import Image
import h5py

In [ ]:
print(tf.__version__)

In [ ]:
# Preview image Frame
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 4

IMAGE_CHANNELS = 3


SEED_SIZE = 100

IMG_SIZE = 64
NOISE_DIM = 100
BATCH_SIZE = 64
EPOCHS = 200
SAVE_FREQ = 20

cross_entropy = tf.keras.losses.BinaryCrossentropy()

In [ ]:
# images_path = data_dir 
# training_data = []

# # Iterating over the images inside the directory and resizing them using
# # Pillow's resize method.
# print('resizing...')

# for filename in os.listdir(images_path):
#     path = os.path.join(images_path, filename)
#     image = Image.open(path).resize((IMG_SIZE, IMG_SIZE), Image.Resampling.LANCZOS)
    
#     training_data.append(np.asarray(image))

# # training_data = (training_data - 127.5) / 127.5

# print('saving file...')
# os.makedirs(input_dir, exist_ok=True)
# np.save(os.path.join(input_dir, 'training_data.npy'), training_data)

In [ ]:
training_data = []


# Look for saved file to save time loading and processing images between runs
print("Looking for saved binary file...")

if not os.path.isfile(os.path.join(input_dir, 'training_data.npy')):
    print("\n File not found, creating new file...\n")
    print('resizing...')
    
    for filename in os.listdir(data_dir):
        image_path = os.path.join(data_dir, filename)
        if os.path.isfile(image_path):
            image = Image.open(image_path).resize((IMG_SIZE, IMG_SIZE), Image.Resampling.LANCZOS)
            image_array = np.array(image)
            training_data.append(image_array)

    training_data = np.array(training_data, dtype=np.float32)
    training_data = (training_data - 127.5) / 127.5 # Normalize to [-1 , 1]

    print("Saving dataset binary file...")
    os.makedirs(input_dir, exist_ok=True)
    np.save(os.path.join(input_dir, 'training_data.npy'), training_data)  # Save processed images as npy file
else:
    print("Data found, loading..")
    training_data = np.load(os.path.join(input_dir, 'training_data.npy'))

print("Dataset length: ", len(training_data))

In [ ]:
# training_data = np.load(os.path.join(input_dir, 'training_data.npy'))

In [ ]:
# images_path = data_dir
# training_data = []
# for filename in os.listdir(images_path):
#     path = os.path.join(images_path, filename)
#     image = Image.open(path).resize((IMG_SIZE, IMG_SIZE), Image.Resampling.LANCZOS)
#     training_data.append(np.asarray(image))
# training_data = np.reshape(training_data, (-1, IMG_SIZE, IMG_SIZE, IMAGE_CHANNELS))
# training_data = training_data / 127.5 - 1

In [ ]:
# Discriminator model
def build_discriminator():
    model = models.Sequential()
    model.add(layers.Conv2D(64, kernel_size=5, strides=2, input_shape=(64, 64, 3), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4)) 

    model.add(layers.Conv2D(128, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization(momentum=0.9))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))  

    model.add(layers.Conv2D(256, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization(momentum=0.9))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))  

    model.add(layers.Conv2D(512, kernel_size=5, strides=2, padding='same'))
    model.add(layers.BatchNormalization(momentum=0.9))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.4))  
    
    model.add(layers.Flatten())
    model.add(layers.Dense(1))
    model.add(layers.Activation('sigmoid'))
    return model

In [ ]:
# Generator model
def build_generator():
    model = models.Sequential()
    model.add(layers.Dense(4*4*512,activation="relu",input_dim=SEED_SIZE)) #64x64 units
    model.add(layers.Reshape((4,4,512)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha = 0.2))

    model.add(layers.Conv2DTranspose(512, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha =0.3))

    model.add(layers.Conv2DTranspose(256, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha = 0.2))

    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha =0.3))
    
    model.add(layers.Conv2DTranspose(32, (5, 5), strides=(2, 2), padding='same', use_bias=False  ))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU(alpha = 0.2))

    model.add(layers.Conv2DTranspose(IMAGE_CHANNELS, (5, 5), strides=(1, 1), padding='same', use_bias=True, activation='tanh'))
    
    return model

In [ ]:
# Generator loss
def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

In [ ]:
# Discriminator loss
def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

In [ ]:
# Train step
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, SEED_SIZE])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

    return gen_loss, disc_loss

In [ ]:
# Training loop
def train(dataset, epochs, checkpoint, checkpoint_prefix):
    for epoch in range(epochs):
        start = time.time()
        for image_batch in dataset:
            t =  train_step(image_batch)
            gen_loss_list.append(t[0])
            disc_loss_list.append(t[1])

        g_loss = sum(gen_loss_list) / len(gen_loss_list)
        d_loss = sum(disc_loss_list) / len(disc_loss_list)

        if (epoch + 1) % SAVE_FREQ == 0:
            save_images(epoch + 1, generator)
            checkpoint.save(file_prefix=checkpoint_prefix)

        print('Epoch {}, Generator Loss: {:.4f}, Discriminator Loss: {:.4f}, Time: {:.2f} sec'.format(epoch + 1, g_loss, d_loss, time.time()-start))
        # Save the model 
    if epoch == EPOCHS - 1:
        generator_file = '/kaggle/working/model/dcgan_generator.h5'
        discriminator_file = '/kaggle/working/model/dcgan_discriminator.h5'
        # Remove existing files if they exist
        try:
            os.remove(generator_file)
            os.remove(discriminator_file)
        except FileNotFoundError:
            pass
        os.makedirs('/kaggle/working/model', exist_ok=True)
        # Lưu mô hình generator và discriminator
        generator.save(generator_file)
        discriminator.save(discriminator_file)

In [ ]:
# def save_images(epoch, generator):
#     noise = tf.random.normal([16, NOISE_DIM])
#     generated_images = generator(noise, training=False)

#     fig = plt.figure(figsize=(4, 4))
#     for i in range(generated_images.shape[0]):
#         plt.subplot(4, 4, i+1)
#         plt.imshow((generated_images[i] + 1) / 2)
#         plt.axis('off')
#     plt.savefig(f'generated_image_epoch_{epoch}.png')
#     plt.close(fig)

In [ ]:
def save_images(epoch, generator, output_dir=output_dir):
    noise = tf.random.normal([16, NOISE_DIM])
    generated_images = generator(noise, training=False)

    fig = plt.figure(figsize=(10, 10))

    for i in range(generated_images.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(generated_images[i, :, :, :] * 0.5 + 0.5)
        plt.axis('off')

    plt.suptitle(f"Generated Images - Epoch {epoch}", fontsize=16)
    plt.savefig(os.path.join(output_dir, f"generated_images_epoch_{epoch}.png"))
    plt.show()

In [ ]:
def display_generated_image(generator, seed_size):
    # Create a random seed for generating new images
    random_seed = tf.random.normal([1, seed_size])

    # Generate an image using the generator
    new_image = generator(random_seed, training=False)

    # Convert the generated image to numpy array
    generated_image = new_image.numpy()

    # Rescale the pixel values to be between 0 and 255
    generated_image = 0.5 * generated_image + 0.5
    generated_image = (generated_image * 255).astype(np.uint8)

    # Plot the generated image
    plt.imshow(generated_image[0])
    plt.axis('off')
    plt.title('Generated Image')
    plt.show()

    return generated_image

In [ ]:
def classify_generated_image(discriminator, generated_image):
    # Use the discriminator to classify the generated image
    discriminator_output = discriminator.predict(generated_image)

    # If the discriminator's output is closer to 1, then it's likely a real image
    if discriminator_output >= 0.5:
        print("The generated image is likely real.")
    else:
        print("The generated image is likely fake.")

In [ ]:
# generator_path = '/kaggle/working/model/generator.h5'
# if os.path.exists(generator_path):
#     os.remove(generator_path)

# discriminator_path = '/kaggle/working/model/discriminator.h5'
# if os.path.exists(discriminator_path):
#     os.remove(discriminator_path)


In [ ]:
# Optimizers
generator_optimizer = Adam(1.5e-4, beta_1=0.5)
discriminator_optimizer = Adam(1.5e-4, beta_1=0.5)

In [ ]:
# Generator and discriminator models
generator = build_generator()
generator.compile(loss='binary_crossentropy', optimizer=generator_optimizer)
generator.summary()

In [ ]:
discriminator = build_discriminator()
discriminator.compile(loss='binary_crossentropy', optimizer=discriminator_optimizer)
discriminator.summary()

In [ ]:
# Load and preprocess data using tf.data.Dataset
training_dataset = tf.data.Dataset.from_tensor_slices(training_data).shuffle(5000).batch(BATCH_SIZE)

In [ ]:
def visualize_random_art(dataset):
    # Shuffle the dataset
    dataset = dataset.shuffle(buffer_size=len(dataset))

    # Take a batch of images from the dataset
    images = next(iter(dataset))

    # Plot the images
    fig = plt.figure(figsize=(12, 12))
    for i in range(1, 37):
        plt.subplot(6, 6, i)
        plt.imshow(images[i])
        plt.axis('off')
    plt.show()


visualize_random_art(training_dataset)

In [ ]:
!mkdir ./training_checkpoints
# os.makedirs('/kaggle/working/training_checkpoints', exist_ok=True)
# checkpoint
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt")
checkpoint = tf.train.Checkpoint(generator_optimizer=generator_optimizer,
                                 discriminator_optimizer=discriminator_optimizer,
                                 generator=generator,
                                 discriminator=discriminator)

In [ ]:
# Train the GAN
gen_loss_list = []
disc_loss_list = []
os.makedirs(output_dir, exist_ok=True)
# checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))
train(training_dataset, EPOCHS, checkpoint, checkpoint_prefix)

In [ ]:
# Plot loss vs epoch
plt.figure(figsize=(8, 6))
plt.plot(range(len(gen_loss_list)), gen_loss_list, label='Generator Loss')
plt.plot(range(len(disc_loss_list)), disc_loss_list, label='Discriminator Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('GAN Loss')
plt.show()

In [ ]:
output_dir = "/kaggle/working/output_model"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Lưu kiến trúc mô hình dưới dạng JSON
model_json = generator.to_json()
json_path = os.path.join(output_dir, "model_architecture.json")
with open(json_path, "w") as json_file:
    json_file.write(model_json)

# Lưu trọng số mô hình
weights_path = os.path.join(output_dir, "model_weights.weights.h5")
generator.save_weights(weights_path)

In [ ]:
generator = tf.keras.models.load_model('/kaggle/working/model/dcgan_generator.h5')
# Display a generated image
generated_image = display_generated_image(generator, SEED_SIZE)

In [ ]:

for i in range(1, 10):
    generated_image = display_generated_image(generator, SEED_SIZE)


In [ ]:
discriminator = tf.keras.models.load_model('/kaggle/working/model/dcgan_discriminator.h5')
# Classify a generated image
classify_generated_image(discriminator, generated_image)

In [ ]:
import os
import numpy as np
import tensorflow as tf
import cv2
from scipy.linalg import sqrtm
# from tensorflow.keras.applications.inception_v3 import InceptionV3, preprocess_input
from PIL import Image
import keras
from keras.applications.inception_v3 import InceptionV3, preprocess_input

In [ ]:
def preprocess_images(images):
    images_resized = np.array([np.array(Image
                                        .fromarray((img * 255)
                                        .astype(np.uint8))
                                        .resize((299, 299))) for img in images])
    images_preprocessed = preprocess_input(images_resized)
    return images_preprocessed

In [ ]:
def inception_score(images, batch_size=32):
    inception_model = InceptionV3(include_top=True, weights='imagenet')
    processed_images = preprocess_images(images)
    preds = inception_model.predict(processed_images, batch_size=batch_size)
    preds = np.exp(preds) / np.sum(np.exp(preds), axis=1, keepdims=True)
    scores = np.sum(preds * np.log(preds), axis=1)
    scores = np.exp(np.mean(scores))
    return scores

In [ ]:
def calculate_fid(real_images, generated_images):
    inception_model = InceptionV3(include_top=False, pooling='avg', input_shape=(299, 299, 3))
    real_images = preprocess_images(real_images)
    generated_images = preprocess_images(generated_images)
    real_activations = inception_model.predict(real_images)
    generated_activations = inception_model.predict(generated_images)
    mu_real, sigma_real = np.mean(real_activations, axis=0), np.cov(real_activations, rowvar=False)
    mu_gen, sigma_gen = np.mean(generated_activations, axis=0), np.cov(generated_activations, rowvar=False)
    diff_mean = np.sum((mu_real - mu_gen)**2)
    cov_mean = sqrtm(sigma_real.dot(sigma_gen))
    if np.iscomplexobj(cov_mean):
        cov_mean = cov_mean.real
    fid = diff_mean + np.trace(sigma_real + sigma_gen - 2*cov_mean)
    return fid

In [ ]:
def generate_fake_samples(generator, seed_size, n_samples):
    generated_images = []
    for _ in range(n_samples):
        random_seed = tf.random.normal([1, seed_size])
        new_image = generator(random_seed, training=False)
        generated_images.append(new_image[0].numpy())
    return generated_images

In [ ]:
def evaluate_GAN(generator, seed_size, n_samples, real_images):
    fake_samples = generate_fake_samples(generator, seed_size, n_samples)
    score = inception_score(fake_samples)
    fid = calculate_fid(real_images, fake_samples)
    return score, fid

In [ ]:
def load_images_from_directory(image_dir):
    images = []
    count = 0
    for filename in os.listdir(image_dir):
        if count >= 150:  # Số lượng ảnh tối đa
            break
        img_path = os.path.join(image_dir, filename)
        if os.path.isfile(img_path):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Chuyển đổi từ BGR sang RGB
            img = img / 255.0  # Chuẩn hóa giá trị ảnh từ 0-255 về 0-1
            images.append(img)
            count += 1
    return images

In [ ]:
validation_image_dir = "../input/wikiart/Realism"
n_samples = 150
real_images = load_images_from_directory(validation_image_dir)
score, fid = evaluate_GAN(generator, SEED_SIZE, n_samples, real_images)
print("Inception Score:", score)
print("Fréchet Inception Distance:", fid)

In [ ]:
model_path = "/kaggle/input/dcgan_model/tensorflow2/dcganggenerator/1/dcgan_generator.keras"
reload_model = tf.keras.models.load_model(model_path)

In [ ]:
for i in range(1, 10):
    generated_image = display_generated_image(generator, SEED_SIZE)

In [ ]:
output_dir = "/kaggle/working/output_model"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

In [ ]:
# Lưu kiến trúc mô hình dưới dạng JSON
model_json = reload_model.to_json()
json_path = os.path.join(output_dir, "model_architecture.json")
with open(json_path, "w") as json_file:
    json_file.write(model_json)

# Lưu trọng số mô hình
weights_path = os.path.join(output_dir, "model_weights.weights.h5")
reload_model.save_weights(weights_path)